In [ ]:
import copy
import itertools
import functools
import collections
import operator
import sys
import re
import numpy as np
import math

In [ ]:
from helpers.functions import *

Configuration

In [ ]:
DIR = "data/2019/"
load_day = functools.partial(load, DIR)

In [ ]:
def intcode_computer(software, idx=0, relative_basis=0, inputs=None, memory=None):

    inputs = list(inputs or [])
    software_size = len(software)
        
    if memory is None:
        memory = collections.defaultdict(int)

    while software[idx] != 99:
        instruction = '{:0>5}'.format(software[idx])
        # print(f"Instruction: {instruction}", end=": ")
        
        modes = instruction[2::-1]
        assert(all([mod in ["0", "1", "2"] for mod in modes]))
        
        opcode = software[idx] % 100
        
        # position, relative or immediate mode
        if opcode in [1, 2, 4, 5, 6, 7, 8, 9]:
            if modes[0] == "1":
                left_idx = idx+1
            elif modes[0] == "2":
                left_idx = software[idx+1] + relative_basis
            else:
                left_idx = software[idx+1]
            
            if left_idx < 0:
                raise ValueError(f"Negative indexing: {left_idx}")
            elif left_idx >= software_size:
                left_term = memory[left_idx]
            else:
                left_term = software[left_idx]
        
        if opcode in [1, 2, 5, 6, 7, 8, 9]:
            if modes[1] == "1":
                right_idx = idx+2
            elif modes[1] == "2":
                right_idx = software[idx+2] + relative_basis
            else:
                right_idx = software[idx+2]

            if right_idx < 0:
                raise ValueError(f"Negative indexing: {right_idx}")
            elif right_idx >= software_size:
                right_term = memory[right_idx]
            else:
                right_term = software[right_idx]
        
        writting_idx = None
        writting_data = None

        if opcode in [1, 2, 7, 8]:
            writting_idx = 3
        elif opcode == 3:
            writting_idx = 1
        
        if writting_idx is not None:
            if modes[writting_idx - 1] == "2":
                writting_idx = software[idx + writting_idx] + relative_basis
            else:
                writting_idx = software[idx + writting_idx]

        if opcode == 1:
            writting_data = left_term + right_term
            idx += 4
        elif opcode == 2:
            writting_data = left_term * right_term
            idx += 4
        elif opcode == 3:
            if len(inputs):
                writting_data = int(inputs.pop(0))
                idx += 2
            else:
                return {"error": "input", "idx": idx, "relative_basis": relative_basis}
        elif opcode == 4:
            return {"output": left_term, "idx": idx + 2, "relative_basis": relative_basis}
        elif opcode == 5:
            if left_term:
                idx = right_term
            else:
                idx += 3
        elif opcode == 6:
            if not left_term:
                idx = right_term
            else:
                idx += 3
        elif opcode == 7:
            writting_data = int(left_term < right_term)
            idx += 4
        elif opcode == 8:
            writting_data = int(left_term == right_term)
            idx += 4
        elif opcode == 9:
            relative_basis += left_term
            idx += 2
        else:
            raise ValueError("error, code:", software[idx], " position: ", idx)
        
        if writting_data is not None:
            if writting_idx < software_size:
                software[writting_idx] = writting_data
            else:
                memory[writting_idx] = writting_data

# Problems

## Day 1

http://adventofcode.com/2019/day/1

In [ ]:
content = load_day(1)

In [ ]:
def fuel(mass):
    return mass // 3 - 2

__Part 1__

In [ ]:
total = 0

for elem in map(int, content):
    total += fuel(elem)
    
print(f'Answer 1: {total}')

__Part 2__

In [ ]:
total = 0

for elem in map(int, content):
    fuel_mass = fuel(elem)
    while fuel_mass > 0:
        total += fuel_mass
        fuel_mass = fuel(fuel_mass)

print(f'Answer 2: {total}')

## Day 2

http://adventofcode.com/2019/day/2

In [ ]:
content = load_day(2)
content = list(map(int, content[0].split(',')))

__Part 1__

In [ ]:
def write_software(noun=12, verb=2):
    software = copy.deepcopy(content)
    software[1:3] = [noun, verb]
    return software

In [ ]:
software = write_software()
intcode_computer(software)
print(f'Anwser 1: {software[0]}')

__Part 2__

In [ ]:
for noun in range(100):
    for verb in range(100):
        software = write_software(noun, verb)
        intcode_computer(software)
        if software[0] == 19690720:
            print('Answer 2: {}'.format(noun * 100 + verb))

## Day 3

http://adventofcode.com/2019/day/3

In [ ]:
content = load_day(3)
ex1 = ["R75,D30,R83,U83,L12,D49,R71,U7,L72", "U62,R66,U55,R34,D71,R55,D58,R83"]
ex2 = ["R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51", "U98,R91,D20,R16,D67,R40,U7,R15,U6,R7"]

In [ ]:
content = [ c.split(',') for c in content]
ex1 = [ c.split(',') for c in ex1]
ex2 = [ c.split(',') for c in ex2]

In [ ]:
def order_to_coords(value, from_coord):
    orient = value[0]
    length = int(value[1:])
    
    if orient == 'R':
        return [(from_coord[0] + x, from_coord[1]) for x in range(1, length + 1)]
    elif orient == 'L':
        return [(from_coord[0] - x, from_coord[1]) for x in range(1, length + 1)]
    elif orient == 'U':
        return [(from_coord[0], from_coord[1] + x) for x in range(1, length + 1)]
    elif orient == 'D':
        return [(from_coord[0], from_coord[1] - x) for x in range(1, length + 1)]

__Part 1__

In [ ]:
paths = []

for row in content:
    cur_pos = (0, 0)
    path = []

    for order in row:
        points = order_to_coords(order, cur_pos)
        path += points
        
        cur_pos = points[-1]
        
    paths += [path]

In [ ]:
intersections = set(paths[0]).intersection(set(paths[1]))
sorted_intersections = sorted(intersections, key=lambda x: abs(x[0]) + abs(x[1]))
print('Answer 1: {}'.format(abs(sorted_intersections[0][0]) + abs(sorted_intersections[0][1])))

__Part 2__

In [ ]:
distances = []
for inter in intersections:
    distances += [2 + paths[0].index(inter) + paths[1].index(inter)] # initial step for each path is not considered
    
print('Answer 2: {}'.format(min(distances)))

## Day 4

http://adventofcode.com/2019/day/4

In [ ]:
# content = load_day(4)
puzzle_input = "145852-616942"

__Part 1__

In [ ]:
def check_input(value):
    if not isinstance(value, str):
        raise ValueError("input should be string")
        
    elems = list(value)
    
    prev_elem = None
    before_elem = None
    
    adjacent = False
    decrease = False
    
    for e in elems:
        if prev_elem is not None:
            if not adjacent:
                adjacent = prev_elem == e
            if not decrease:
                decrease = ord(prev_elem) > ord(e)

        prev_elem = e
    return adjacent and not decrease

In [ ]:
candidates = []
for v in range(*map(int, puzzle_input.split('-'))):
    inp = str(v)
    candidates += [inp] if check_input(inp) else []
print(f'Answer 1: {len(candidates)}')

__Part 2__

In [ ]:
def lengths(value):
    le = []
    char = ''
    count = 1
    for v in value[1:]:
        if v == char:
            count += 1
        else:
            le.append(count)
            count = 1
            
        char = v
    return 

In [ ]:
for c in candidates:
    
    total += int(check_input2(str(v)))
print(f'Answer 2: {total}')

## Day 5

http://adventofcode.com/2019/day/5

In [ ]:
content = load_day(5)
content = list(map(int, content[0].split(',')))

__Part 1__

In [ ]:
def pass_all_tests(software, inputs):
    memory = collections.defaultdict(int)
    idx = 0
    relative_basis = 0

    while software[idx] != 99:
        outputs = intcode_computer(software, idx=idx, relative_basis=relative_basis, inputs=inputs, memory=memory)
        # print(outputs["output"])
        idx = outputs["idx"]
        relative_basis = outputs["relative_basis"]

    return outputs["output"]

In [ ]:
software = copy.deepcopy(content)
print(f"Answer 1: {pass_all_tests(software, [1])}")

__Part 2__

In [ ]:
software = copy.deepcopy(content)
print(f"Answer 2: {pass_all_tests(software, [5])}")

## Day 6

http://adventofcode.com/2019/day/6

In [ ]:
content = load_day(6)

In [ ]:
children = collections.defaultdict(list)
parents = {}

for from_obj, to_obj in map(lambda l: l.split(')'), content):
    children[from_obj] += [to_obj]
    parents[to_obj] = from_obj

__Part 1__

In [ ]:
queue = ['COM']
scores = {'COM': 0}

while queue:
    obj = queue.pop(0)
    for child in children[obj]:
        scores[child] = scores[obj] + 1
        queue += [child]

print(f"Answer 1: {sum(scores.values())}")

__Part 2__

In [ ]:
def get_path(obj):
    path = []
    while obj != 'COM':
        path += [obj]
        obj = parents[obj]
    return path + ["COM"]

In [ ]:
you_path = get_path("YOU")
san_path = get_path("SAN")

common_stars = set(you_path) & set(san_path)

for idx in range(len(you_path)):
    if you_path[idx] in common_stars:
        you_length = idx
        break

for idx in range(len(san_path)):
    if san_path[idx] in common_stars:
        san_length = idx
        break

print(f"Answer 2: {you_length + san_length - 2}")  # first and last "transfers" do not count

## Day 7

http://adventofcode.com/2019/day/7

In [ ]:
content = load_day(7)
content = list(map(int, content[0].split(',')))

__Part 1__

In [ ]:
max_output = 0

for perm in map(list, itertools.permutations(range(5))):
    input_val = 0

    while perm:
        software = copy.deepcopy(content)
        inputs = [perm.pop(0), input_val]
        outputs = intcode_computer(software, idx=0, inputs=inputs)
        input_val = outputs["output"]

    if input_val > max_output:
        max_output = input_val
    
print(f"Anwser 1: {max_output}")

__Part 2__

In [ ]:
max_output = 0

for perm in map(list, itertools.permutations(range(5, 10))):
    softwares = [content] * 5
    pointers = [0] * 5
    outputs = {}

    ampli_idx = 0
    input_val = 0
    
    stored_value = 0
    
    while True:
        inputs = [input_val]

        if perm:
            inputs.insert(0, perm.pop(0))

        outputs = intcode_computer(softwares[ampli_idx], idx=pointers[ampli_idx], inputs=inputs)

        if softwares[ampli_idx][pointers[ampli_idx]] != 99:
            input_val = outputs["output"]
            
            pointers[ampli_idx] = outputs["idx"]

            if ampli_idx == 4:
                stored_value = input_val

            ampli_idx = (ampli_idx + 1) % 5
        else:
            break

    if stored_value > max_output:
        max_output = stored_value
print(f"Anwser 2: {max_output}")

## Day 8

http://adventofcode.com/2019/day/8

In [ ]:
content = load_day(8)[0]
width, height = 25, 6

__Part 1__

In [ ]:
img_size = width * height
N = len(content)

images = []

min_zero = img_size
result = 0

for idx in range(0, N, img_size):
    layer = list(content[idx:idx+img_size])
    counter = collections.Counter(layer)
    
    if counter['0'] < min_zero:
        min_zero = counter['0']
        result = counter['1'] * counter['2']
    
    images += [ np.array(layer) ]
        
print(f"Answer 1: {result}")

__Part 2__

In [ ]:
result_image = np.empty(img_size, dtype="str")

for img in images[::-1]:
    result_image[img == "1"] = "x"
    result_image[img == "0"] = " "
    
res = result_image.tolist()
print("\n".join([''.join(res[width*i:width*(i+1)]) for i in range(height)]))

## Day 9

http://adventofcode.com/2019/day/9

In [ ]:
content = load_day(9)
content = list(map(int, content[0].split(',')))

__Part 1__

In [ ]:
software = copy.deepcopy(content)
print(f"Answer 1: {pass_all_tests(software, [1])}")

__Part 2__

In [ ]:
software = copy.deepcopy(content)
print(f"Answer 2: {pass_all_tests(software, [2])}")

## Day 10

http://adventofcode.com/2019/day/10

In [ ]:
content = load_day(10)
positions = np.where(np.array(list(map(list, content)))=="#")
points = list(zip(positions[1], positions[0]))  # points are (x, y), increasing to the right and the bottom

nb_points = len(points)

__Part 1__

In [ ]:
max_dirs = 0
location_idx = None

for i_idx in range(nb_points):
    source = points[i_idx]
    directions = set()

    for j_idx in range(nb_points):
        dest = points[j_idx]
        orient = [dest[0] - source[0], dest[1] - source[1]]

        if orient != [0, 0]:
            if orient[0] == 0 or orient[1] == 0:
                denom = abs(orient[0] + orient[1])
            else:
                denom = math.gcd(abs(orient[0]), abs(orient[1]))

            directions.add((orient[0]//denom, orient[1]//denom))
        # print(source, dest, directions)

    if len(directions) > max_dirs:
        location_idx = i_idx
        max_dirs = len(directions)

print(f"Anwser 1: {max_dirs}")

__Part 2__

In [ ]:
def compute_angle(source, target):
    x = target[0] - source[0]
    y = target[1] - source[1]
    return math.atan2(-y, x)  # y goes to down

def l2_dist(source, target):
    return math.sqrt((source[0]-target[0]) ** 2 + (source[1]-target[1]) ** 2)

def find_quadrant(x):
    if x <= math.pi/2 and x >= 0:
        return 1
    elif x > math.pi/2:
        return 3
    else:
        return 2

def rotation_compare(x, y):
    quadrant_x = find_quadrant(x)
    quadrant_y = find_quadrant(y)
    
    if quadrant_x != quadrant_y:
        return quadrant_x - quadrant_y
    else:
        return y - x
    
def cmp_to_key(mycmp):
    'Convert a cmp= function into a key= function'
    class K(object):
        def __init__(self, obj, *args):
            self.obj = obj
        def __lt__(self, other):
            return mycmp(self.obj, other.obj) < 0
        def __gt__(self, other):
            return mycmp(self.obj, other.obj) > 0
        def __eq__(self, other):
            return mycmp(self.obj, other.obj) == 0
        def __le__(self, other):
            return mycmp(self.obj, other.obj) <= 0
        def __ge__(self, other):
            return mycmp(self.obj, other.obj) >= 0
        def __ne__(self, other):
            return mycmp(self.obj, other.obj) != 0
    return K

In [ ]:
# Checking ordering on angles
assert(rotation_compare(math.pi/2, math.pi/4) < 0)
assert(rotation_compare(math.pi/3, math.pi/6) < 0)
assert(rotation_compare(math.pi/3, 0) < 0)
assert(rotation_compare(0, -math.pi/4) < 0)
assert(rotation_compare(-math.pi/6, -math.pi/4) < 0)
assert(rotation_compare(-math.pi/3, -math.pi/2) < 0)
assert(rotation_compare(-math.pi/2, -3*math.pi/4) < 0)
assert(rotation_compare(-4*math.pi/6, -3*math.pi/4) < 0)
assert(rotation_compare(-3*math.pi/4, math.pi) < 0)
assert(rotation_compare(math.pi, 3*math.pi/4) < 0)
assert(rotation_compare(3*math.pi/4, 4*math.pi/6) < 0)
assert(rotation_compare(4*math.pi/6, 0.000001+math.pi/2) < 0)

In [ ]:
angles = collections.defaultdict(list)
location = points[location_idx]
idx_of_interest = 200

for j_idx in range(nb_points):
    if j_idx != location_idx:
        angles[compute_angle(location, points[j_idx])].append(points[j_idx])

for (angle, pts) in angles.items():
    angles[angle] = sorted(pts, key=lambda x: l2_dist(location, x))

ordered_angles = sorted(angles.keys(), key=cmp_to_key(rotation_compare))

nb_rounds = (idx_of_interest - 1) // max_dirs
angle_idx = (idx_of_interest - 1) % max_dirs
answer = angles[ordered_angles[angle_idx]][nb_rounds]

print(f"Answer 2: {answer[0] * 100 + answer[1]}")

## Day 11

http://adventofcode.com/2019/day/11

In [ ]:
content = load_day(11)
content = list(map(int, content[0].split(',')))

__Part 1__

In [ ]:
def delta_from_orientation(orientation):    
    if orientation == 0:
        return (0, 1)
    elif orientation == 1:
        return (1, 0)
    elif orientation == 2:
        return (0, -1)
    elif orientation == 3:
        return (-1, 0)

In [ ]:
grid = collections.defaultdict(int)  # keys are (x, y)
position = (0, 0)
orientation = 0  # maps {0, 1, 2, 3} to ["T", "L", "B", "R"]
path = set()

software = copy.deepcopy(content)
memory = collections.defaultdict(int)
idx = 0
relative_basis = 0
inputs = [1, 1]  # for initial input

while True:
    if inputs:  # 2nd run
        inputs.pop()
    else:  # 1st run
        inputs = [grid[position]]

    outputs = intcode_computer(software, idx=idx, relative_basis=relative_basis, inputs=inputs, memory=memory)
    if outputs is None:
        break

    path.add(position)
    
    if inputs:  # 1sr run
        grid[position] = outputs["output"]
    else:  # 2nd run: 0 -> L, 1 -> R
        orientation = (orientation + (-1) ** outputs["output"]) % 4
       
        delta = delta_from_orientation(orientation)
        position = (position[0] + delta[0], position[1] + delta[1])

    idx = outputs["idx"]
    relative_basis = outputs["relative_basis"]

print(f"Answer 1: {len(path)}")

__Part 2__

In [ ]:
min_x = min(map(lambda x: x[0], grid.keys()))
max_x = max(map(lambda x: x[0], grid.keys()))
min_y = min(map(lambda x: x[1], grid.keys()))
max_y = max(map(lambda x: x[1], grid.keys()))

max_x -= min_x
max_y -= min_y

drawing = np.zeros((max_y + 1, max_x + 1))
for key, val in grid.items():
    drawing[key[1] - min_y][key[0] - min_x] = val
print('\n'.join([''.join(map(lambda x: '#' if x else ' ', line)) for line in drawing[::-1, ::-1].tolist()]))

## Day 12

http://adventofcode.com/2019/day/12

In [ ]:
content = load_day(12)

__Part 1__

In [ ]:
positions = np.array(list(map(lambda line: list(map(lambda elem: int(elem.split('=')[1]), line[1:-1].split(','))), content)))
velocities = np.zeros(positions.shape, dtype=int)

def compute_system_energy():
    return (np.abs(positions).sum(axis=1) * np.abs(velocities).sum(axis=1)).sum()

for step in range(1000):
    for idx in range(positions.shape[0]):
        velocities += np.where(positions>positions[idx], -1, 0) + np.where(positions<positions[idx], 1, 0)
    positions += velocities

print(f"Answer 1: {compute_system_energy()}")

__Part 2__

In [ ]:
positions = np.array(list(map(lambda line: list(map(lambda elem: int(elem.split('=')[1]), line[1:-1].split(','))), content)))
velocities = np.zeros(positions.shape, dtype=int)

cycles_length = []

for dim in range(3):
    history = set()
    step = 0
    
    pos = positions[:, dim]
    vel = velocities[:, dim]
    
    while True:
        if step > 0:
            for idx in range(positions.shape[0]):
                vel += np.where(pos>pos[idx], -1, 0) + np.where(pos<pos[idx], 1, 0)
            pos += vel

        item = str(pos.tolist() + vel.tolist())
        if item in history:
            cycles_length.append(step)
            break
        else:
            history.add(item)
            step += 1

print(f"Answer 2: {np.lcm.reduce(cycles_length)}")

## Day 13

http://adventofcode.com/2019/day/13

In [ ]:
content = load_day(13)
content = list(map(int, content[0].split(',')))

__Part 1__

In [ ]:
software = copy.deepcopy(content)
memory = collections.defaultdict(int)

idx = 0
relative_basis = 0
grid_instructions = []

grid = collections.defaultdict(int)

while True:
    outputs = intcode_computer(software, idx=idx, relative_basis=relative_basis, memory=memory)

    if outputs is None or "error" in outputs:
        break

    grid_instructions += [outputs["output"]]
    idx = outputs["idx"]
    relative_basis = outputs["relative_basis"]
        
    if len(grid_instructions) == 3:
        grid[(grid_instructions[0], grid_instructions[1])] = grid_instructions[2]
        grid_instructions = []

print(f"Answer 1: {collections.Counter(grid.values())[2]}")

__Part 2__

In [ ]:
MAPPING_BLOCKS = [' ', '#', 'W', '_', 'u']

def print_grid(software, score):
    print(f"Score: {score}")
    for row in range(23):
        print(''.join(map(lambda x: MAPPING_BLOCKS[x], software[639 + 42 * row:639 +42*(row+1)])))

In [ ]:
software = copy.deepcopy(content)
memory = collections.defaultdict(int)
software[0] = 2

idx = 0
relative_basis = 0
grid_instructions = []

grid = collections.defaultdict(int)

while software[idx] != 99:
    outputs = intcode_computer(software, idx=idx, relative_basis=relative_basis, inputs=inputs, memory=memory)
    inputs = []

    if outputs:
        if "error" in outputs:
            if collections.Counter(grid.values())[2]:
                # print_grid(software, grid[(-1, 0)])
                inputs = [last_ball_x_pos - last_pad_x_pos]
            else:
                break

        else:
            grid_instructions += [outputs["output"]]
        
            if len(grid_instructions) == 3:
                grid[(grid_instructions[0], grid_instructions[1])] = grid_instructions[2]
                
                if grid_instructions[2] == 3:
                    last_pad_x_pos = grid_instructions[0]
                elif grid_instructions[2] == 4:
                    last_ball_x_pos = grid_instructions[0]
                grid_instructions = []

        idx = outputs["idx"]
        relative_basis = outputs["relative_basis"]

print(f"Answer 2: {grid[(-1, 0)]}")

## Day 14

http://adventofcode.com/2019/day/14

In [ ]:
content = load_day(14)

ingredients_for = dict()
receipts_with = collections.defaultdict(list)

for row in content:
    source, target = row.split(' => ')
    nb_t, kind_t = target.strip().split(' ')
    sources = []
    for elem in source.split(','):
        nb_s, kind_s = elem.strip().split(' ')
        sources.append([int(nb_s), kind_s])
        receipts_with[kind_s].append(kind_t)
    ingredients_for[kind_t] = [int(nb_t), sources]
    
heights = collections.defaultdict(int)
queue = ['ORE']

while queue:
    elem = queue.pop(0)
    for ing in receipts_with[elem]:
        heights[ing] = heights[elem] + 1
        queue.append(ing)

heights = dict(sorted(heights.items(), key=lambda x: -x[1]))

__Part 1__

In [ ]:
requirements = collections.defaultdict(int)
requirements['FUEL'] = 1

for ing in heights.keys():
    if ing == 'ORE':
        break
    
    receipt = ingredients_for[ing]

    needed = requirements.pop(ing)    
    times = int(math.ceil(needed / receipt[0]))
        
    for req in receipt[1]:
        requirements[req[1]] += req[0] * times

cost_per_unit = requirements['ORE']
print(f"Answer 1: {cost_per_unit}")

__Part 2__

In [ ]:
wastes = collections.defaultdict(int)

space = 1000000000000
units = 0

while True:
    requirements = collections.defaultdict(int)
    
    new_units = space // cost_per_unit
    requirements['FUEL'] = new_units

    for ing in heights.keys():
        if ing == 'ORE':
            break

        receipt = ingredients_for[ing]

        needed_for_receipt = requirements.pop(ing)
        taken_from_wastes = min(wastes[ing], needed)
        needed_to_produce = needed_for_receipt - taken_from_wastes
        wastes[ing] -= taken_from_wastes
        
        times = int(math.ceil(needed_to_produce / receipt[0]))
        produced = receipt[0] * times
        wastes[ing] += produced - needed_to_produce

        for req in receipt[1]:
            requirements[req[1]] += req[0] * times
    
    if new_units and requirements['ORE'] < space:
        space -= requirements['ORE']
        units += new_units
    else:
        break

print(f"Answer 2: {units}")

## Day 15

http://adventofcode.com/2019/day/15

In [ ]:
content = load_day(15)

__Part 1__

__Part 2__

## Day 16

http://adventofcode.com/2019/day/16

In [ ]:
content = load_day(16)

__Part 1__

__Part 2__

## Day 17

http://adventofcode.com/2019/day/17

In [ ]:
content = load_day(17)

__Part 1__

__Part 2__

## Day 18

http://adventofcode.com/2019/day/18

In [ ]:
content = load_day(18)

__Part 1__

__Part 2__

## Day 19

http://adventofcode.com/2019/day/19

In [ ]:
content = load_day(19)

__Part 1__

__Part 2__

## Day 20

http://adventofcode.com/2019/day/20

In [ ]:
content = load_day(20)

__Part 1__

__Part 2__

## Day 21

http://adventofcode.com/2019/day/21

In [ ]:
content = load_day(21)

__Part 1__

__Part 2__

## Day 22

http://adventofcode.com/2019/day/22

In [ ]:
content = load_day(22)

__Part 1__

__Part 2__

## Day 23

http://adventofcode.com/2019/day/23

In [ ]:
content = load_day(23)

__Part 1__

__Part 2__

## Day 24

http://adventofcode.com/2019/day/24

In [ ]:
content = load_day(24)

__Part 1__

__Part 2__

## Day 25

http://adventofcode.com/2019/day/25

In [ ]:
content = load_day(25)

__Part 1__

__Part 2__